In [58]:
import openmc 
import openmc.mgxs as mgxs
import numpy as np

In [59]:
uoc_9 = openmc.Material()
uoc_9.set_density('atom/b-cm', 7.038386E-2)
uoc_9.add_nuclide('U235', 2.27325e-3)
uoc_9.add_nuclide('U238', 2.269476e-2)
uoc_9.add_nuclide('O16', 3.561871e-2)
uoc_9.add_nuclide('C0', 9.79714e-3)
uoc_9.temperature = 900

graphite = openmc.Material()
graphite.set_density("g/cc", 1.8)
graphite.add_nuclide("C0", 9.025164e-2)
graphite.temperature = 900

mats = openmc.Materials([uoc_9, graphite])
mats.export_to_xml()

sphere = openmc.Sphere(r=6.3849)
fuel = openmc.Cell(fill=uoc_9, region=-sphere, cell_id=1, name="fuel")
outer_sphere = openmc.Sphere(r=8, boundary_type="reflective")
moderator = openmc.Cell(fill=graphite, region=+sphere & -outer_sphere, cell_id=2, name="moderator")
geom = openmc.Geometry([fuel, moderator])
geom.export_to_xml()

settings = openmc.Settings()
settings.batches = 50
settings.inactive = 10
settings.particles = 5000
settings.temperature = {"multipole": True, "method": "interpolation"}
settings.export_to_xml()

/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Cell instance already exists with id=1.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Cell instance already exists with id=2.
  warn(msg, IDWarning)


In [63]:
def generate_openmc_tallies_xml(energy_groups, delayed_groups, domains, domain_ids, tallies_file):
    groups = mgxs.EnergyGroups()
    groups.group_edges = np.array(energy_groups)
    big_group = mgxs.EnergyGroups() 
    big_energy_group = [energy_groups[0], energy_groups[-1]]
    big_group.group_edges = np.array(big_energy_group)
    energy_filter = openmc.EnergyFilter(energy_groups)
    domain_dict = {} 
    for name in domain_ids:
        domain_dict[name] = {} 
    for domain, name in zip(domains,domain_ids): 
        domain_dict[name]["beta"] = mgxs.Beta(domain=domain, energy_groups=big_group, delayed_groups=delayed_groups, name=str(name)+"_beta")
        domain_dict[name]["chi"] = mgxs.Chi(domain=domain, groups=groups, name=str(name)+"_chi")
        domain_dict[name]["chidelayed"] = mgxs.ChiDelayed(domain=domain, energy_groups=groups, name=str(name)+"_chidelayed")
        domain_dict[name]["decayrate"] = mgxs.DecayRate(domain=domain, energy_groups=big_group, delayed_groups=delayed_groups, name=str(name)+"_decayrate")
        domain_dict[name]["scatterprobmatrix"] = mgxs.ScatterProbabilityMatrix(domain=domain, groups=groups, name=str(name)+"_scatterprobmatrix")
        domain_dict[name]["inversevelocity"] = mgxs.InverseVelocity(domain=domain, groups=groups, name=str(name)+"_inversevelocity")
        domain_dict[name]["fissionxs"] = mgxs.FissionXS(domain=domain, groups=groups, name=str(name)+"_fissionxs")
        domain_dict[name]["tally"] = openmc.Tally(name=str(name)+" tally")
        domain_dict[name]["filter"] = openmc.CellFilter(domain)
        domain_dict[name]["tally"].filters = [domain_dict[name]["filter"], energy_filter]
        domain_dict[name]["tally"].scores = ["kappa-fission", "nu-fission", "absorption", "scatter"]
        tallies_file += domain_dict[name]["beta"].tallies.values()
        tallies_file += domain_dict[name]["chi"].tallies.values()
        tallies_file += domain_dict[name]["chidelayed"].tallies.values()
        tallies_file += domain_dict[name]["decayrate"].tallies.values()
        tallies_file += domain_dict[name]["scatterprobmatrix"].tallies.values()
        tallies_file += domain_dict[name]["inversevelocity"].tallies.values()
        tallies_file += domain_dict[name]["fissionxs"].tallies.values()
        tallies_file.append(domain_dict[name]["tally"])
    tallies_file.export_to_xml()
    return domain_dict

In [64]:
tallies_file = openmc.Tallies()
domain_dict = generate_openmc_tallies_xml([1e-8, 1, 1e3, 1e5, 20.0e7], list(range(1,7)), [fuel, moderator], [1,2], tallies_file)

/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=91.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=92.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=95.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=93.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=104.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance

In [ ]:
tallies_file = openmc.Tallies()
groups = mgxs.EnergyGroups()
delayed_groups = list(range(1,7))
energy_groups = [1e-8, 1, 1e3, 1e5, 20.0e7]
groups.group_edges = np.array(energy_groups)
big_group = mgxs.EnergyGroups() 
big_energy_group = [1e-8, 20.0e7]
big_group.group_edges = np.array(big_energy_group)
energy_filter = openmc.EnergyFilter(energy_groups)
fuel_cell_filter = openmc.CellFilter(fuel)

fuel_beta = mgxs.Beta(domain=fuel, energy_groups=big_group, delayed_groups=delayed_groups, name="fuel_beta")
fuel_chi = mgxs.Chi(domain=fuel, groups=groups, name="fuel_chi")
fuel_chidelayed = mgxs.ChiDelayed(domain=fuel, energy_groups=groups, name="fuel_chidelayed")
fuel_decayrate = mgxs.DecayRate(domain=fuel, energy_groups=big_group, delayed_groups=delayed_groups, name="fuel_decayrate")
#fuel_diffusioncoefficient = mgxs.DiffusionCoefficient(domain=fuel, groups=groups, name="fuel_diffusioncoefficient")
fuel_spm = mgxs.ScatterProbabilityMatrix(domain=fuel, groups=groups, name="fuel_spm")
fuel_iv = mgxs.InverseVelocity(domain=fuel, groups=groups, name="fuel_iv")
fuel_fxs = mgxs.FissionXS(domain=fuel, groups=groups, name="fuel_fxs")
fuel_tally = openmc.Tally(name="fuel tally")
fuel_tally.filters = [fuel_cell_filter, energy_filter]
fuel_tally.scores = ["kappa-fission", "nu-fission", "absorption", "scatter"]

tallies_file += fuel_beta.tallies.values()
tallies_file += fuel_chi.tallies.values()
tallies_file += fuel_chidelayed.tallies.values()
tallies_file += fuel_decayrate.tallies.values()
#tallies_file += fuel_diffusioncoefficient.tallies.values()
tallies_file += fuel_spm.tallies.values()
tallies_file += fuel_iv.tallies.values()
tallies_file += fuel_fxs.tallies.values()
tallies_file.append(fuel_tally)
tallies_file.export_to_xml()

In [ ]:
# openmc.mgxs.Beta [cm^-1]
# openmc.mgxs.Chi [%]
# openmc.mgxs.ChiDelayed [cm^-1]
# openmc.mgxs.DecayRate [cm^-1]
# openmc.mgxs.DiffusionCoefficient()
# kappa-fission * settings.batches * settings.particles
# mgxs.ScatterProbabilityMatrix [cm^-1]
# nu-fission * settings.batches * settings.particles
# mgxs.InverseVelocity [second/cm]
# mgxs.FissionXS [cm^-1]

In [65]:
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [66]:
sp = openmc.StatePoint('statepoint.50.h5')
for k in sp.filters: 
    v = sp.filters[k]
    if isinstance(v, openmc.filter.CellFilter):
        print(v.bins)

[2]
[1]


/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=104.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=111.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=91.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=92.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=93.
  warn(msg, IDWarning)
/Users/gwenchee/anaconda3/envs/openmc-env/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instanc

In [48]:
sp.filters

{16: EnergyFilter
 	Values         =	[1.e-08 1.e+00 1.e+03 1.e+05 2.e+08]
 	ID             =	16,
 23: CellFilter
 	Bins           =	[2]
 	ID             =	23,
 3: CellFilter
 	Bins           =	[1]
 	ID             =	3,
 4: EnergyFilter
 	Values         =	[1.e-08 2.e+08]
 	ID             =	4,
 5: DelayedGroupFilter
 	Bins           =	[1 2 3 4 5 6]
 	ID             =	5,
 7: EnergyoutFilter
 	Values         =	[1.e-08 1.e+00 1.e+03 1.e+05 2.e+08]
 	ID             =	7}

In [ ]:
#fuel_beta.load_from_statepoint(sp)
#fuel_beta.get_pandas_dataframe()

In [61]:
#CHIT 
fuel_chi.load_from_statepoint(sp)
fuel_chi.get_pandas_dataframe()

NameError: name 'fuel_chi' is not defined

In [ ]:
fuel_chidelayed.load_from_statepoint(sp)
fuel_chidelayed.get_pandas_dataframe()

In [ ]:
fuel_decayrate.load_from_statepoint(sp)
fuel_decayrate.get_pandas_dataframe()

In [ ]:
fuel_tally = sp.get_tally(name='fuel tally')
df_fuel_tally = fuel_tally.get_pandas_dataframe()
df_fuel_tally = df_fuel_tally[df_fuel_tally["score"]== "kappa-fission"]
df_fuel_tally["fisse"] = df_fuel_tally["mean"] * sp.current_batch * sp.n_particles
df_fuel_tally

In [ ]:
fuel_spm.load_from_statepoint(sp)
fuel_spm.get_pandas_dataframe()

In [ ]:
fuel_tally = sp.get_tally(name='fuel tally')
df_fuel_tally = fuel_tally.get_pandas_dataframe()
df_fuel_tally = df_fuel_tally[df_fuel_tally["score"]== "nu-fission"]
df_fuel_tally["nsf"] = df_fuel_tally["mean"] * sp.current_batch * sp.n_particles
df_fuel_tally

In [ ]:
fuel_iv.load_from_statepoint(sp)
fuel_iv.get_pandas_dataframe()

In [ ]:
fuel_tally = sp.get_tally(name='fuel tally')
df_fuel_tally = fuel_tally.get_pandas_dataframe()
df_fuel_tally = df_fuel_tally[df_fuel_tally["score"]== "nu-fission"]
df_fuel_tally["nsf"] = df_fuel_tally["mean"] * sp.current_batch * sp.n_particles
df_fuel_tally

In [ ]:
fuel_tally = sp.get_tally(name='fuel tally')
df_fuel_tally = fuel_tally.get_pandas_dataframe()
df_fuel_tally_scatter = df_fuel_tally[df_fuel_tally["score"]== "scatter"]
df_fuel_tally_abs = df_fuel_tally[df_fuel_tally["score"]== "absorption"]
df_fuel_tally = df_fuel_tally[df_fuel_tally["score"]== "absorption"]
df_fuel_tally["remxs"] = df_fuel_tally_scatter["mean"] + df_fuel_tally_abs["mean"]
df_fuel_tally

In [ ]:
df_fuel_tally_scatter["mean"] + df_fuel_tally_abs["mean"]

In [ ]:
for i,j in zip(df_fuel_tally_abs["mean"],df_fuel_tally_scatter["mean"]):
    print(i+j)

In [56]:
fuel_fxs.load_from_statepoint(sp)
fuel_fxs.get_pandas_dataframe()

NameError: name 'fuel_fxs' is not defined

In [74]:
domain_dict[1]["fissionxs"].load_from_statepoint(sp)
list(domain_dict[1]["fissionxs"].get_pandas_dataframe()["mean"])

[0.005326609754292178,
 0.006819051198892783,
 0.03619849595844361,
 0.1901923841024031]

In [77]:
def get_tallies_list(sp, tally):
    tally.load_from_statepoint(sp)
    return list(tally.get_pandas_dataframe()["mean"])

In [78]:
get_tallies_list(sp, domain_dict[1]["beta"])

[0.00022765434528523683,
 0.00128348757224532,
 0.0012872662856220486,
 0.0031286071367546357,
 0.0015376581809872095,
 0.0006353818673912366]